<h4>Lily Grumbach
    <br>M1 Humanités numériques - Université PSL</h4>
<h1><center>Rendu TAIS-TAL </center></h1>
<h2><center>Partie TAIS</center></h2>
<h3><center>1/3 : Harmonisation des données Géopolitiques pour le fond de carte de 1914</center></h3>

<b><u>PLAN:</u></b>
<h4>1) Harmonisation des données Géopolitiques pour le fond de carte de 1914</h4>
<br>
    
2) Reconstruire et adapter la base de l'IREL aux données à disposition 
<br/>
    
3) Géoréférencer les données trouvées


In [2]:
import csv
import pandas as pd
import numpy as np
import matplotlib as plt

## Import des CSV : 

**Données du GeoJson:**
* Historical National Boundaries 1914 map  

**Données historique des statuts géopolitiques des territoires**
* GPH entities over time : 
* GPH entites avec lien wikidata pour la désambiguisation 

In [3]:
#IMPORTER LES CSV

#GPH entities over time
df_geopol = pd.read_csv("./GeoPolHist-202103/medialab-GeoPolHist-fb19b66/data/GeoPolHist_entities_status_over_time.csv")
df_geopol = df_geopol.rename(columns={"GPH_name":"QGIS_PlaceName"})
    
#SHP file QGIS 1914
df_qgis = pd.read_csv('./HistNatBoundaries/HistNatBoundaries_map1914.csv')

## Isoler de la liste GPH overtime des territoires ayant existé en 1914

In [4]:
#isoler les territoires qui ont existé en 1914 depuis GPH status
listGPH1914 = []
for i in range (len(df_geopol)):
    if df_geopol.iloc[i,2] != "?" and df_geopol.iloc[i,3] != "?":
        if 1914 in range(int(df_geopol.iloc[i,2]),int(df_geopol.iloc[i,3])): #ne prendre que les possessions qui ont existé en 1914
            if int(df_geopol.iloc[i,2]) == 1914 : #On retire les possessions qui débutent en 1914
                pass
            else:
                listGPH1914.append(df_geopol.iloc[i,])

df_1914GPH= pd.DataFrame(listGPH1914)
df_1914GPH.to_csv("./output/1-QGIS_GPH_HistNatBound/14GPH.csv")

## Isoler les pays souverains
pour l'ensemble des GPH

In [5]:
#Faire un dictionnaire des codes de pays :
dict_sovereign = {}
for i in range (len(df_1914GPH)):
    statut = df_1914GPH.iloc[i,4]
    dict_sovereign[int(df_1914GPH.iloc[i,0])]=df_1914GPH.iloc[i,1]

# print(dict_sovereign)

#En faire un df
df_1914SovereignGPH = pd.DataFrame.from_dict(data=dict_sovereign,orient="index")

df_1914SovereignGPH.rename(columns={0:"NAME"})

# df_1914SovereignGPH.to_csv("../Sovereigncountries.csv")

,NAME
2,United States of America
3,Alaska
4,Hawaii
5,Virgin Islands
6,Puerto Rico
...,...
9983,South Shetland Is.
9984,South Sandwich Is.
9993,Bouvet I.
9994,Ross Dependency


### Ajout au df HistNatBound les informations de GPH

### Fusion des df "généaral"et "restricted" (seulement avec les noms qui apparaîssent dans le df de HistNatBoundaries)

In [6]:
#Fusionner les df de qgis et GPH pour avoir une liste unie des noms.
df_14merge = pd.merge(df_1914GPH,df_qgis, how='outer',on='QGIS_PlaceName')

df_14merge.to_csv("./output/1-QGIS_GPH_HistNatBound/1914_ensemble_GPH_HistNatBound_Merged.csv")


#Fusion restreinte (càd seulement les noms de lieux qui apparaissent dans les deux) : 
df_14restrictedmerge = pd.merge(df_1914GPH,df_qgis, how='inner',on='QGIS_PlaceName')
df_14restrictedmerge.to_csv("./output/1-QGIS_GPH_HistNatBound/1914_restricted_GPH_HistNatBound_Merged.csv")

df_14restrictedmerge #94 lignes
df_14merge #

,GPH_code,QGIS_PlaceName,start_year,end_year,GPH_status,sovereign_GPH_code
0,2.0,United States of America,1816,2020.0,Sovereign,NaN
1,3.0,Alaska,1867,1959.0,Colony of,2.0
2,4.0,Hawaii,1898,1960.0,Colony of,2.0
3,5.0,Virgin Islands,1816,1917.0,Colony of,390.0
4,6.0,Puerto Rico,1898,1952.0,Colony of,2.0
...,...,...,...,...,...,...
1137,NaN,Turkey,NaN,NaN,NaN,NaN
1138,NaN,United States,NaN,NaN,NaN,NaN
1139,NaN,US Virgin Islands,NaN,NaN,NaN,NaN
1140,NaN,Wake Island,NaN,NaN,NaN,NaN


In [9]:
#remplissage du dfpar itération :
seriePlaceName = df_14restrictedmerge["QGIS_PlaceName"]
# print(seriePlaceName.values)
l_notInDf = []

#création d'une liste avec le nom de tous les lieux n'apparaissant pas dansle df_restrictedmerged
for i in range (len(df_qgis)):
    nomlieu = df_qgis.iloc[i,0]
    if nomlieu not in seriePlaceName.values:
        l_notInDf.append(nomlieu)
l_notInDf

['Abyssinia',
 'Anglo-Egyptian Sudan',
 'Antarctica',
 'Arabia',
 'Ascension Island',
 'Austria - Hungary ',
 'Azores',
 'Baker Island',
 'Bechuanaland',
 'Belgian Congo ',
 'Bouvet Island',
 'British Burma',
 'British East Africa',
 'British Guyana',
 'British Honduras',
 'British Malaya',
 'British Somaliland',
 'British Virgin Islands',
 'Cabo Verde',
 'Cameroons',
 'Canary Island',
 'Caroline Islands',
 'Caroline Islands',
 'Cayman Islands',
 'Ceylon',
 'Chagos Islands',
 'China',
 'Chosen (Korea)',
 'Christmas Island',
 'Cocos Islands',
 'Colombia',
 'Cook Islands',
 'Cyprus',
 'Dominican Republic',
 'Dominion of Canada',
 'Dutch East Indies',
 'Dutch Guiana',
 'Egypt',
 'Ellice Island',
 'Falkland Islands',
 'Faroe Islands',
 'Formosa (Japan)',
 'French equatorial Africa',
 'French Guiana',
 'French Guinea',
 'French Indo China',
 'French Somaliland',
 'French Southern Territories',
 'French Sudan',
 'German East Africa',
 'German New Guinea',
 'German Samoa Island ',
 'German So

In [11]:
#création d'un df à part avec la liste l_notInDf
na = np.arange(len(l_notInDf))
columns=['GPH_code', 'QGIS_PlaceName', 'start_year', 'end_year', 'GPH_status',
       'sovereign_GPH_code']

df_append = pd.DataFrame(index=na,columns=columns)
df_append["QGIS_PlaceName"]=l_notInDf

#Concaténation au df voulu
df_1914GPHxQgis = pd.concat([df_14restrictedmerge,df_append])

for i in range(len(df_1914GPHxQgis)):
    sovCode= df_1914GPHxQgis.iloc[i,5]
    
    if np.isnan(sovCode):
#         print("oups")
        df_1914GPHxQgis.iloc[i,5]=df_1914GPHxQgis.iloc[i,0]



In [12]:
df_1914GPHxQgis

,GPH_code,QGIS_PlaceName,start_year,end_year,GPH_status,sovereign_GPH_code
0,6,Puerto Rico,1898,1952,Colony of,2.0
1,10,Greenland,1816,1953,Colony of,390.0
2,30,Bermuda,1816,2020,Colony of,200.0
3,31,Bahamas,1816,1973,Colony of,200.0
4,40,Cuba,1906,2020,Sovereign,40.0
...,...,...,...,...,...,...
113,NaN,Turkey,NaN,NaN,NaN,NaN
114,NaN,United States,NaN,NaN,NaN,NaN
115,NaN,US Virgin Islands,NaN,NaN,NaN,NaN
116,NaN,Wake Island,NaN,NaN,NaN,NaN


In [13]:
## Exportation
df_1914GPHxQgis=df_1914GPHxQgis.rename(columns={"QGIS_PlaceName":"NAME"})
df_1914GPHxQgis.to_csv("./output/1-QGIS_GPH_HistNatBound/1914-final_GPHxQgis.csv",index=False)
# df_1914GPHxQgis

## Fusionner les annotations manuelles avec les nouveaux apports de df

Dans MANUELdf_1914GPHxQgis, nous avons complété les lieux avec leur satut correspondant dans la base GPH ainsi qu'attribué, lecas échéant, le nom de l'Etat dominant le lieu mentionné.

In [15]:
df_Manuel914GPHQGIS = pd.read_csv("./output_finaux/MANUELdf_1914GPHxQgis.csv")    
df_Manuel914GPHQGIS

,GPH_code,NAME,start_year,end_year,GPH_status,sovereign_GPH_code
0,530.0,Abyssinia,1898.0,1936.0,Sovereign,NaN
1,681.0,Aden,1838.0,1962.0,Colony of,200.0
2,700.0,Afghanistan,1879.0,1919.0,Protectorate of,200.0
3,339.0,Albania,1912.0,1939.0,Sovereign,NaN
4,232.0,Andorra,1816.0,1993.0,Sovereign (unrecognized),NaN
...,...,...,...,...,...,...
198,165.0,Uruguay,1828.0,2020.0,Sovereign,NaN
199,5.0,US Virgin Islands,1816.0,1917.0,Colony of,390.0
200,101.0,Venezuela,1830.0,2020.0,Sovereign,NaN
201,9720.0,Wake Island,1898.0,1941.0,Colony of,2.0


In [16]:
#Ajouter une colonne df_MANUEL avec le sovereign_GPH_code des pays souverains
df_Manuel914GPHQGIS['SovereignGPH_code'] = df_Manuel914GPHQGIS.apply(lambda _: '', axis=1)

for i in range(len(df_Manuel914GPHQGIS)):
    sovCode= df_Manuel914GPHQGIS.iloc[i,5]
    
    if np.isnan(sovCode):
#         print("oups")
        df_Manuel914GPHQGIS.iloc[i,5]=df_Manuel914GPHQGIS.iloc[i,0]
        df_Manuel914GPHQGIS.iloc[i,6]=df_Manuel914GPHQGIS.iloc[i,0]


In [17]:
df_Manuel914GPHQGIS

,GPH_code,NAME,start_year,end_year,GPH_status,sovereign_GPH_code,SovereignGPH_code
0,530.0,Abyssinia,1898.0,1936.0,Sovereign,530.0,530
1,681.0,Aden,1838.0,1962.0,Colony of,200.0,
2,700.0,Afghanistan,1879.0,1919.0,Protectorate of,200.0,
3,339.0,Albania,1912.0,1939.0,Sovereign,339.0,339
4,232.0,Andorra,1816.0,1993.0,Sovereign (unrecognized),232.0,232
...,...,...,...,...,...,...,...
198,165.0,Uruguay,1828.0,2020.0,Sovereign,165.0,165
199,5.0,US Virgin Islands,1816.0,1917.0,Colony of,390.0,
200,101.0,Venezuela,1830.0,2020.0,Sovereign,101.0,101
201,9720.0,Wake Island,1898.0,1941.0,Colony of,2.0,


In [13]:
df_Manuel914GPHQGIS.to_csv("./output_finaux/df_1914manuelCarto.csv",index=False)

# Exploration des df créés

#### En fonction du statut des GPE

In [14]:
#Décompte statut des pays en fct° du df restreint ou non :

GPHStatus14 = df_14merge.groupby(by="GPH_status").size()
restrictedGPHStatus14  = df_14restrictedmerge.groupby(by="GPH_status").size()

#création d'un df de comparaison : 
df_comparaisonGPHstatuts = pd.concat([restrictedGPHStatus14,GPHStatus14],axis=1)
df_comparaisonGPHstatuts = df_comparaisonGPHstatuts.rename(columns = {0:"Liste restreinte",1:"Ensemble"})
df_comparaisonGPHstatuts

,Liste restreinte,Ensemble
Colony of,25.0,96
Occupied by,1.0,9
Part of,14.0,629
Protectorate of,6.0,85
Sovereign,35.0,46
Sovereign (unrecognized),6.0,14
Claimed by,NaN,24
Dependency of,NaN,3
Discovered,NaN,9
Dissolved into,NaN,3


#### En fonction de la dépendance

In [15]:
#Estimer le nombre de noms de pays communs : 
for i in range (len(df_1914GPH)):
    statut = df_1914GPH.iloc[i,4]
    if statut == "Sovereign":
        GPHSovereign = df_14merge.groupby(by="sovereign_GPH_code").size()
        restrictedGPHSovereign = df_14restrictedmerge.groupby(by="sovereign_GPH_code").size()

df_comparaisonGPHsovereign = pd.concat([restrictedGPHSovereign,GPHSovereign],axis=1)
df_comparaisonGPHsovereign = df_comparaisonGPHsovereign.rename(columns = {0:"Liste restreinte",1:"Ensemble"})

#Attribuer à chaque GPHCode le nom de la souveraineté dans le df:
df_comparaisonGPHsovereign.index = df_comparaisonGPHsovereign.index.astype(int)

for i in range(len(df_comparaisonGPHsovereign)):
    code = df_comparaisonGPHsovereign.index[i]
    if code in dict_sovereign.keys():
        df_comparaisonGPHsovereign = df_comparaisonGPHsovereign.rename({code:dict_sovereign[code]})
    else :
        print(code)
#Ordonner en fonction de l'occurence dans Liste Restreinte        
df_comparaisonGPHsovereign = df_comparaisonGPHsovereign.sort_values("Liste restreinte",ascending=False)
df_comparaisonGPHsovereign.head(15)

345
352
626
651
912
5011
5796
9810


,Liste restreinte,Ensemble
sovereign_GPH_code,,
United Kingdom,21.0,138
United States of America,4.0,39
Netherlands Antilles,3.0,4
France,3.0,34
Italy,2.0,8
Denmark,2.0,5
Portugal,1.0,11
Russia (USSR),1.0,52
St. Kitts-Nevis,1.0,3


In [16]:
df_comparaisonGPHsovereign

,Liste restreinte,Ensemble
sovereign_GPH_code,,
United Kingdom,21.0,138
United States of America,4.0,39
Netherlands Antilles,3.0,4
France,3.0,34
Italy,2.0,8
...,...,...
Wallis and Futuna Is.,NaN,3
Society Is.,NaN,1
Pitcairn Is.,NaN,1
